# Задание А


| TID | Items          |
|-----|----------------|
| 100 | {a, c, d}      |
| 200 | {b, d, f}      |
| 300 | {a, b, d, e}   |
| 400 | {b, c, e, g}   |
| 500 | {a, c, d, e}   |
| 600 | {b, d, f, g}   |
| 700 | {c, f}         |

Минимальная поддержка: 2
Минимальная достоверность (confidence): 0.5

# Подготовка данных

In [ ]:
from itertools import combinations
from collections import defaultdict
import pandas as pd

transactions = {
    100: {"a", "c", "d"},
    200: {"b", "d", "f"},
    300: {"a", "b", "d", "e"},
    400: {"b", "c", "e", "g"},
    500: {"a", "c", "d", "e"},
    600: {"b", "d", "f", "g"},
    700: {"c", "f"},
}

minsup = 2
minconf = 0.5

# Apriori с выводом Ck и Lk

In [ ]:
def apriori_steps(transactions, minsup):
    def get_support(candidates):
        support_count = defaultdict(int)
        for items in transactions.values():
            for candidate in candidates:
                if set(candidate).issubset(items):
                    support_count[candidate] += 1
        return support_count

    k = 1
    L_prev = []
    all_frequent = dict()
    while True:
        if k == 1:
            items = sorted(set(i for t in transactions.values() for i in t))
            candidates = [tuple([item]) for item in items]
        else:
            items = sorted(set(i for l in L_prev for i in l))
            candidates = list(combinations(items, k))

        Ck = set(tuple(sorted(c)) for c in candidates)
        support_count = get_support(Ck)
        print(f"\nC{k} (Кандидаты):")
        for itemset, support in sorted(support_count.items()):
            print(f"{itemset}: {support}")

        Lk = {itemset: support for itemset, support in support_count.items() if support >= minsup}
        if not Lk:
            break

        print(f"\nL{k} (Частые наборы):")
        for itemset, support in sorted(Lk.items()):
            print(f"{itemset}: {support}")

        all_frequent.update(Lk)
        L_prev = list(Lk.keys())
        k += 1
    return all_frequent

frequent_itemsets = apriori_steps(transactions, minsup)


C1 (Кандидаты):
('a',): 3
('b',): 4
('c',): 4
('d',): 5
('e',): 3
('f',): 3
('g',): 2

L1 (Частые наборы):
('a',): 3
('b',): 4
('c',): 4
('d',): 5
('e',): 3
('f',): 3
('g',): 2

C2 (Кандидаты):
('a', 'b'): 1
('a', 'c'): 2
('a', 'd'): 3
('a', 'e'): 2
('b', 'c'): 1
('b', 'd'): 3
('b', 'e'): 2
('b', 'f'): 2
('b', 'g'): 2
('c', 'd'): 2
('c', 'e'): 2
('c', 'f'): 1
('c', 'g'): 1
('d', 'e'): 2
('d', 'f'): 2
('d', 'g'): 1
('e', 'g'): 1
('f', 'g'): 1

L2 (Частые наборы):
('a', 'c'): 2
('a', 'd'): 3
('a', 'e'): 2
('b', 'd'): 3
('b', 'e'): 2
('b', 'f'): 2
('b', 'g'): 2
('c', 'd'): 2
('c', 'e'): 2
('d', 'e'): 2
('d', 'f'): 2

C3 (Кандидаты):
('a', 'b', 'd'): 1
('a', 'b', 'e'): 1
('a', 'c', 'd'): 2
('a', 'c', 'e'): 1
('a', 'd', 'e'): 2
('b', 'c', 'e'): 1
('b', 'c', 'g'): 1
('b', 'd', 'e'): 1
('b', 'd', 'f'): 2
('b', 'd', 'g'): 1
('b', 'e', 'g'): 1
('b', 'f', 'g'): 1
('c', 'd', 'e'): 1
('c', 'e', 'g'): 1
('d', 'f', 'g'): 1

L3 (Частые наборы):
('a', 'c', 'd'): 2
('a', 'd', 'e'): 2
('b', 'd', 'f'): 

# Генерация ассоциативных правил

In [ ]:
def generate_rules_verbose(frequent_itemsets, minconf):
    rules = []
    for itemset in frequent_itemsets:
        if len(itemset) >= 2:
            support = frequent_itemsets[itemset]
            for i in range(1, len(itemset)):
                for antecedent in combinations(itemset, i):
                    consequent = tuple(sorted(set(itemset) - set(antecedent)))
                    if antecedent in frequent_itemsets:
                        conf = support / frequent_itemsets[antecedent]
                        if conf >= minconf:
                            rules.append({
                                'Rule': f"{antecedent} → {consequent}",
                                'Support': support,
                                'Confidence': round(conf, 2)
                            })
    return pd.DataFrame(rules).sort_values(by='Confidence', ascending=False)

generate_rules_verbose(frequent_itemsets, minconf)

,Rule,Support,Confidence
33,"('d', 'e') → ('a',)",2,1.00
32,"('a', 'e') → ('d',)",2,1.00
3,"('a',) → ('d',)",3,1.00
27,"('b', 'f') → ('d',)",2,1.00
23,"('c', 'd') → ('a',)",2,1.00
21,"('a', 'c') → ('d',)",2,1.00
28,"('d', 'f') → ('b',)",2,1.00
18,"('g',) → ('b',)",2,1.00
7,"('b',) → ('d',)",3,0.75
29,"('a',) → ('d', 'e')",2,0.67


# Алгоритм FP-Growth

In [ ]:
from collections import Counter

# Подсчёт глобальной частоты
item_counter = Counter()
for items in transactions.values():
    item_counter.update(items)

# Убираем элементы с поддержкой < minsup
frequent_items = {item for item, count in item_counter.items() if count >= minsup}
sorted_items = sorted(frequent_items, key=lambda x: (-item_counter[x], x))

# Преобразуем транзакции: фильтрация и сортировка
def sort_transaction(trans, order):
    return [item for item in order if item in trans]

sorted_transactions = [sort_transaction(t, sorted_items) for t in transactions.values()]
for tid, t in zip(transactions, sorted_transactions):
    print(f"TID {tid}: {t}")


TID 100: ['d', 'c', 'a']
TID 200: ['d', 'b', 'f']
TID 300: ['d', 'b', 'a', 'e']
TID 400: ['b', 'c', 'e', 'g']
TID 500: ['d', 'c', 'a', 'e']
TID 600: ['d', 'b', 'f', 'g']
TID 700: ['c', 'f']
